In [8]:
from pathlib import Path

import pandas as pd
from tqdm import tqdm

In [3]:
seed = 23 # 23 42 24666
init_df_path = Path(f"../../data/classification/baseline/train_{seed}.csv")
output_folder = Path(f"../../data/classification/back_translation")
text_column = "Текст"
label_column = "Тональность"

In [4]:
df = pd.read_csv(init_df_path)
print(df.shape)


texts, labels = [], []
texts.extend(df[text_column].tolist())
labels.extend(df[label_column].tolist())

(11668, 4)


In [ ]:
# df_done = pd.read_csv(output_folder / f"train_{seed}.csv")
# print(df_done.shape)

# texts.extend(df_done[text_column])
# labels.extend(df_done[label_column])

# GPT ParaPhraser

In [5]:
# pip install git+https://github.com/RussianNLP/russian_paraphrasers@master

In [6]:
from russian_paraphrasers import GPTParaphraser

paraphraser = GPTParaphraser(model_name="gpt2", range_cand=False, make_eval=False)

In [13]:
fails = 0

for i, raw in tqdm(df.iterrows()):
    
    text, label = raw[text_column], raw[label_column]
    try:
        new_text = paraphraser.generate(
            text, n=1, temperature=0.7, 
            top_k=10, top_p=0.9, 
            max_length=50, repetition_penalty=1.5,
            threshold=0.9
        )["results"][0]


        texts.append(new_text)
        labels.append(label)
        
    except:
        print(f"Fail: {text}")
        fails += 1
    
print(f"Failed {fails} times.")
print(len(texts), len(labels))

# MT 5

In [14]:
from russian_paraphrasers import Mt5Paraphraser

In [ ]:
paraphraser = Mt5Paraphraser(model_name="mt5-large", range_cand=False, make_eval=False)

In [ ]:
fails = 0

for i, raw in tqdm(df.iterrows()):
    
    text, label = raw[text_column], raw[label_column]
    try:
        new_text = paraphraser.generate(
            text, n=1, temperature=0.7, 
            top_k=10, top_p=0.9, 
            max_length=50, repetition_penalty=1.5,
            threshold=0.9
        )["results"][0]


        texts.append(new_text)
        labels.append(label)
        
    except:
        print(f"Fail: {text}")
        fails += 1
    
print(f"Failed {fails} times.")
print(len(texts), len(labels))

# Final DF

In [ ]:
df_done = pd.DataFrame()

df_done[text_column] = texts
df_done[label_column] = labels

print(df_done.shape)
df_done.tail(10)

In [ ]:
df_done.to_csv(output_folder / f"train_{seed}.csv", index=False, encoding="utf-8")